# Biased News Generator fine-tuning Notebook
This Notebook fine-tunes the DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit model.
It uses ~700 article + generated promts and also generated Chain-of-Thoughts, in trying to improve the models task
of rewriting a Neutral news article into a more rightw wing biased 

### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture output
import os

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
    !pip install wandb
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps 
    !pip install wandb


### Unsloth
Loading the 8b Param DeepSeek-R1-Distill-Llama. Using unsloth should enable us to do faster and more memory friendly fine-tuning.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth
# unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-1.5B-GGUF
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.3: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [20]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.1, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

TypeError: Unsloth: Your model already has LoRA adapters. Your new parameters are different.

### Weights and Biases for Monitoring of the fine-tuning
Ensuring we are Logged in and then also setting evnvars for eg. the project destination.

In [4]:
# from huggingface_hub import notebook_login

# notebook_login()

import wandb
wandb.login()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yonekintrash (yonekintrash-ludwigshafen-university-of-business-and-society) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="BiasNewsGen-project"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="end"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"



## Accessing our training data 
The data used for the fine-tuning is stored on Hugging Face. It was originally scraped from NUIS a rigth wing news portal.
We also do some preprocessing like dropping fields we dont need merging others. Adding the **crucial \<think>\</think>** markers around the chain-of-thought is now done via the chat pattern.

In [6]:
from datasets import load_dataset

# Load dataset from the correct URL
dataset = load_dataset(
    "Yonekin/nius_reasoning",
    split="train",
    # use_auth_token=True  # Ensure you have permission to access private datasets
)

def modify_entry(entry):
    """Modifies a single dataset entry while preserving its structure."""
    reasoning = entry.pop("reasoning", "")
    reasoning = entry.pop("output_with_reasoning", "")

    if reasoning:
        entry["output"] = f"<think>{reasoning}</think>\n{entry['output']}"

    entry.pop("article_mongodb_id", None)

    return entry

# Apply modifications while keeping the dataset structure
dataset = dataset.map(modify_entry)

# Print dataset information
print(dataset)
print(dataset[0])



Dataset({
    features: ['input', 'output'],
    num_rows: 766
})
{'input': 'Generiere einen politisch rechten Artikel, der die zunehmende Gewalt in deutschen Städten thematisiert und den Vorfall im Einkaufszentrum „Marktforum“ in Rheinhausen hervorhebt, bei dem ein 49-jähriger Mann Passanten mit einem Messer bedrohte. Schildern Sie die Wut und Frustration der Bürger über die Sicherheitslage in städtischen Gebieten und diskutieren Sie die Rolle von psychischen Erkrankungen und den Umgang der Behörden mit solchen Vorfällen. Erwägen Sie, welche Maßnahmen ergriffen werden könnten, um die öffentliche Sicherheit zu verbessern, und integrieren Sie dabei eine kritische Perspektive auf die aktuelle Politik und deren Einfluss auf die Gesellschaft. Berücksichtigen Sie die Reaktionen der Bürger und analysieren Sie, wie kollektive Maßnahmen, wie das Eingreifen der mutigen Passanten, ein Beispiel für zivilen Mut und Verantwortungsbewusstsein sein können.', 'output': '<think>1. Begin by interpreting

One issue is this dataset has multiple columns. For `Ollama` and `llama.cpp` to function like a custom `ChatGPT` Chatbot, we must only have 2 columns - an `instruction` and an `output` column.

In [19]:
print(dataset.column_names)

['input', 'output']


To solve this, we shall do the following:
* Merge all columns into 1 instruction prompt.
* Remember LLMs are text predictors, so we can customize the instruction to anything we like!
* Use the `to_sharegpt` function to do this column merging process!

For example below in our [Titanic CSV finetuning notebook](https://colab.research.google.com/drive/1VYkncZMfGFkeCEgN2IzbZIKEDkyQuJAS?usp=sharing), we merged multiple columns in 1 prompt:

<img src="https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Merge.png" height="100">

To merge multiple columns into 1, use `merged_prompt`.
* Enclose all columns in curly braces `{}`.
* Optional text must be enclused in `[[]]`. For example if the column "Pclass" is empty, the merging function will not show the text and skp this. This is useful for datasets with missing values.
* You can select every column, or a few!
* Select the output or target / prediction column in `output_column_name`. For the Alpaca dataset, this will be `output`.

To make the finetune handle multiple turns (like in ChatGPT), we have to create a "fake" dataset with multiple turns - we use `conversation_extension` to randomnly select some conversations from the dataset, and pack them together into 1 conversation.

In [7]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt="{input}",
    output_column_name="output",
    conversation_extension=1,  # Select more to handle longer conversations
)

from unsloth import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Finally use `standardize_sharegpt` to fix up the dataset!

### Customizable Chat Templates

You also need to specify a chat template. Previously, you could use the Alpaca format as shown below.

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

Now, you have to use `{INPUT}` for the instruction and `{OUTPUT}` for the response.

We also allow you to use an optional `{SYSTEM}` field. This is useful for Ollama when you want to use a custom system prompt (also like in ChatGPT).

You can also not put a `{SYSTEM}` field, and just put plain text.

```python
chat_template = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""
```

Use below if you want to use the Llama-3 prompt format. You must use the `instruct` and not the `base` model if you use this!
```python
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
```

For the ChatML format:
```python
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""
```

The issue is the Alpaca format has 3 fields, whilst OpenAI style chatbots must only use 2 fields (instruction and response). That's why we used the `to_sharegpt` function to merge these columns into 1.

In [17]:
chat_template = """Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
<｜User｜>
{INPUT}
<｜Assistant｜> <think>
{OUTPUT}
</think>
"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/766 [00:00<?, ? examples/s]

In [23]:
dataset[0]


{'conversations': [{'content': "('Generiere einen politisch rechten Artikel, der die zunehmende Gewalt in deutschen Städten thematisiert und den Vorfall im Einkaufszentrum „Marktforum“ in Rheinhausen hervorhebt, bei dem ein 49-jähriger Mann Passanten mit einem Messer bedrohte. Schildern Sie die Wut und Frustration der Bürger über die Sicherheitslage in städtischen Gebieten und diskutieren Sie die Rolle von psychischen Erkrankungen und den Umgang der Behörden mit solchen Vorfällen. Erwägen Sie, welche Maßnahmen ergriffen werden könnten, um die öffentliche Sicherheit zu verbessern, und integrieren Sie dabei eine kritische Perspektive auf die aktuelle Politik und deren Einfluss auf die Gesellschaft. Berücksichtigen Sie die Reaktionen der Bürger und analysieren Sie, wie kollektive Maßnahmen, wie das Eingreifen der mutigen Passanten, ein Beispiel für zivilen Mut und Verantwortungsbewusstsein sein können.',)",
   'role': 'user'},
  {'content': '<think>1. Begin by interpreting the input promp

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 5,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 16, # How many samples will be done before real batch size= grad_ac_steps * batch size
        #gradient_accumulation_steps = 3, #can facilitate bigger batch sizes eventhough memory cant handle them by accumulating batches together
        warmup_ratio = 0.1,
        max_steps = 200,
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-6,  #  controls speed how fast the model learns
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.015,         # to prevent overfitting
        lr_scheduler_type = "linear", # changes LR overtime to avoid local minima
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Tokenizing to ["text"] (num_proc=5):   0%|          | 0/766 [00:00<?, ? examples/s]

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A6000. Max memory = 47.529 GB.
5.748 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 766 | Num Epochs = 5 | Total steps = 200
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/4,670,623,744 (0.90% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
5,2.168700


Unsloth: Will smartly offload gradients to save VRAM!


In [29]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

run.finish()

1018.5109 seconds used for training.
16.98 minutes used for training.
Peak reserved memory = 15.002 GB.
Peak reserved memory for training = 3.82 GB.
Peak reserved memory % of max memory = 31.564 %.
Peak reserved memory for training % of max memory = 8.037 %.


NameError: name 'run' is not defined

<a name="Inference"></a>
### Inference
Let's run the model! Unsloth makes inference natively 2x faster as well! You should use prompts which are similar to the ones you had finetuned on, otherwise you might get bad results!

In [21]:
print("start")
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Generiere einen politisch rechten Artikel, der die zunehmende Gewalt in deutschen Städten thematisiert und den Vorfall im Einkaufszentrum „Marktforum“ in Rheinhausen hervorhebt, bei dem ein 49-jähriger Mann Passanten mit einem Messer bedrohte. Schildern Sie die Wut und Frustration der Bürger über die Sicherheitslage in städtischen Gebieten und diskutieren Sie die Rolle von psychischen Erkrankungen und den Umgang der Behörden mit solchen Vorfällen. Erwägen Sie, welche Maßnahmen ergriffen werden könnten, um die öffentliche Sicherheit zu verbessern, und integrieren Sie dabei eine kritische Perspektive auf die aktuelle Politik und deren Einfluss auf die Gesellschaft. Berücksichtigen Sie die Reaktionen der Bürger und analysieren Sie, wie kollektive Maßnahmen, wie das Eingreifen der mutigen Passanten, ein Beispiel für zivilen Mut und Verantwortungsbewusstsein sein können."},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = False)
output = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 3000, pad_token_id = tokenizer.eos_token_id)
# Konvertiere den Input und Output zurück in lesbaren Text
prompt_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Ausgabe mit guter Formatierung
# print("=== Prompt ===\n")
# print(prompt_text)
# print("\n=== Output ===\n")
# print(output_text)
# generated_ids = model.generate(
#     input_ids,
#     max_new_tokens=8000,
#     pad_token_id=tokenizer.eos_token_id
# )
# text_output = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
# print(text_output)
# print("finsih")

# print("start", flush=True)
# # ...
# print(text_output, flush=True)
# print("finish", flush=True)


start
<｜begin▁of▁sentence｜>Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
<｜User｜>
Generiere einen politisch rechten Artikel, der die zunehmende Gewalt in deutschen Städten thematisiert und den Vorfall im Einkaufszentrum „Marktforum“ in Rheinhausen hervorhebt, bei dem ein 49-jähriger Mann Passanten mit einem Messer bedrohte. Schildern Sie die Wut und Frustration der Bürger über die Sicherheitslage in städtischen Gebieten und diskutieren Sie die Rolle von psychischen Erkrankungen und den Umgang der Behörden mit solchen Vorfällen. Erwägen Sie, welche Maßnahmen ergriffen werden könnten, um die öffentliche Sicherheit zu verbessern, und integrieren Sie dabei eine kritische Perspektive auf die aktuelle Politik und deren Einfluss auf die Gesellschaft. Berücksichtigen Sie die Reaktionen der Bürger und analysieren Sie, wie kollektive Maßnahmen, wie das Eingreifen der mutigen Passanten, ein Beispiel für 

In [23]:
print("sta")
model1, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model1) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Generiere einen politisch rechten Artikel, der die zunehmende Gewalt in deutschen Städten thematisiert und den Vorfall im Einkaufszentrum „Marktforum“ in Rheinhausen hervorhebt, bei dem ein 49-jähriger Mann Passanten mit einem Messer bedrohte. Schildern Sie die Wut und Frustration der Bürger über die Sicherheitslage in städtischen Gebieten und diskutieren Sie die Rolle von psychischen Erkrankungen und den Umgang der Behörden mit solchen Vorfällen. Erwägen Sie, welche Maßnahmen ergriffen werden könnten, um die öffentliche Sicherheit zu verbessern, und integrieren Sie dabei eine kritische Perspektive auf die aktuelle Politik und deren Einfluss auf die Gesellschaft. Berücksichtigen Sie die Reaktionen der Bürger und analysieren Sie, wie kollektive Maßnahmen, wie das Eingreifen der mutigen Passanten, ein Beispiel für zivilen Mut und Verantwortungsbewusstsein sein können."},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = False)
_ = model1.generate(input_ids, streamer = text_streamer, max_new_tokens = 2000, pad_token_id = tokenizer.eos_token_id)

# generated_ids = model.generate(
#     input_ids,
#     max_new_tokens=8000,
#     pad_token_id=tokenizer.eos_token_id
# )
# text_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

sta
==((====))==  Unsloth 2025.3.3: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<｜begin▁of▁sentence｜><｜User｜>Generiere einen politisch rechten Artikel, der die zunehmende Gewalt in deutschen Städten thematisiert und den Vorfall im Einkaufszentrum „Marktforum“ in Rheinhausen hervorhebt, bei dem ein 49-jähriger Mann Passanten mit einem Messer bedrohte. Schildern Sie die Wut und Frustration der Bürger über die Sicherheitslage in städtischen Gebieten und diskutieren Sie die Rolle von psychischen Erkrankungen und den Umgang der Behörden mit solchen Vorfällen. Erwägen Sie, welche Maßnahmen ergriffen werden könnt

Since we created an actual chatbot, you can also do longer conversations by manually adding alternating conversations between the user and assistant!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8"},
    {"role": "assistant", "content": "The fibonacci sequence continues as 13, 21, 34, 55 and 89."},
    {"role": "user",      "content": "What is France's tallest tower called?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

messages = [                    # Change below!
    {"role": "user", "content": "Describe anything special about a sequence. Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

<a name="Ollama"></a>
### Ollama Support

[Unsloth](https://github.com/unslothai/unsloth) now allows you to automatically finetune and create a [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md), and export to [Ollama](https://ollama.com/)! This makes finetuning much easier and provides a seamless workflow from `Unsloth` to `Ollama`!

Let's first install `Ollama`!

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

Next, we shall save the model to GGUF / llama.cpp

We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

We also support saving to multiple GGUF options in a list fashion! This can speed things up by 10 minutes or more if you want multiple export formats!

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

We use `subprocess` to start `Ollama` up in a non blocking fashion! In your own desktop, you can simply open up a new `terminal` and type `ollama serve`, but in Colab, we have to use this hack!

In [ ]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3)  # Wait for a few seconds for Ollama to load!

`Ollama` needs a `Modelfile`, which specifies the model's prompt format. Let's print Unsloth's auto generated one:

In [ ]:
print(tokenizer._ollama_modelfile)

We now will create an `Ollama` model called `unsloth_model` using the `Modelfile` which we auto generated!

In [ ]:
!ollama create unsloth_model -f ./model/Modelfile

And now we can do inference on it via `Ollama`!

You can also upload to `Ollama` and try the `Ollama` Desktop app by heading to https://www.ollama.com/

In [ ]:
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
    }'

# ChatGPT interactive mode

### ⭐ To run the finetuned model like in a ChatGPT style interface, first click the **| >_ |** button.
![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Where_Terminal.png)

---
---
---

### ⭐ Then, type `ollama run unsloth_model`

![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Terminal_Type.png)

---
---
---
### ⭐ And you have a ChatGPT style assistant!

### Type any question you like and press `ENTER`. If you want to exit, hit `CTRL + D`
![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Assistant.png)You can also use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
